In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import vcf
import pysam
import vcfpy

In [6]:
pd.set_option('display.max_rows', 8)

In [7]:
def read_vcf(path):
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    return pd.read_csv(io.StringIO(''.join(lines)), dtype={'#CHROM': str, 'POS':int, 'ID':str, 'REF': str, 'ALT': str, 'QUAL': str, 'FILTER': str, 'INFO': str}, sep='\t').rename(columns={'#CHROM': 'CHROM'})

# Exporting Dataframes for CADD Annotation and Final Annotation Results

In order to annotate the files using CADD, they must be a very small size (2 MB) per file. Therefore, we will have to split up some of the larger dataframes into smaller sets of variants. In addition, we should cut off the last three columns of the dataframes, 'QUAL, FILTER, INFO' because they take up file space and are not needed for the annotations. Once the prior is finished, we need to export the dataframes using the format above with the long header, which we have already done a few times. Once we download them from the Jupyter home to the computer, we must open them in a text editor and cut out the header, but keep the column names. This way, it will work in the CADD tool. 

## HGMD 

### CADD:

Below is the process used to export files for annotation for the non-coding region variants for HGMD for CADD:

In [71]:
hgmd_noncoding_region_variants_only_for_cadd = hgmd_noncoding_region_variants_only.drop(['INFO', 'QUAL', 'FILTER'], axis = 1)
hgmd_noncoding_region_variants_only_for_cadd

,CHROM,POS,ID,REF,ALT
9,chr10,100988295,CM114899,C,T
10,chr10,100988415,CM164756,A,T
11,chr10,100988457,CM127719,C,T
12,chr10,100988526,CM1610318,A,G
...,...,...,...,...,...
64559,chrX,85964053,CS1810957,C,G
64560,chrX,85964054,CS1723659,T,C
64561,chrX,85965588,CS173873,T,C
64562,chrX,85968639,CS032064,A,T


In [72]:
header = """##fileformat=VCFv4.1
##fileDate=20090805
##source=myImputationProgramV3.1
##reference=file:///seq/references/
#CHROM POS ID REF ALT QUAL FILTER INFO
"""

output_VCF = "hgmd_noncoding_for_cadd.vcf"
with open(output_VCF, 'w') as vcf:
    vcf.write(header)

hgmd_noncoding_region_variants_only_for_cadd.to_csv(output_VCF, sep="\t", mode='a', index=False)

Below is the annotated file for the HGMD non-coding region for CADD:

In [32]:
hgmd_noncoding_annotated_cadd = pd.read_table('hgmd_noncoding_noheader.tsv', sep='\t')
hgmd_noncoding_annotated_cadd

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.
/opt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13,120) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,#Chrom,Pos,Ref,Alt,Type,Length,AnnoType,Consequence,ConsScore,ConsDetail,...,Freq10000bp,Rare10000bp,Sngl10000bp,EnsembleRegulatoryFeature,dbscSNV-ada_score,dbscSNV-rf_score,RemapOverlapTF,RemapOverlapCL,RawScore,PHRED
0,1,7961859,C,G,SNV,0,RegulatoryFeature,REGULATORY,4,regulatory,...,26,48,1451,Promoter,NaN,NaN,208.0,475.0,1.043084,12.600
1,1,7961859,C,G,SNV,0,Transcript,INTRONIC,2,intron,...,26,48,1451,Promoter,NaN,NaN,208.0,475.0,1.043084,12.600
2,1,9720021,G,A,SNV,0,RegulatoryFeature,REGULATORY,4,regulatory,...,9,74,1406,CTCF Binding Site,0.00003,0.014,12.0,15.0,-0.385227,0.135
3,1,9720021,G,A,SNV,0,Transcript,SPLICE_SITE,5,"splice,intron",...,9,74,1406,CTCF Binding Site,0.00003,0.014,12.0,15.0,-0.385227,0.135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8609,X,154965965,C,G,SNV,0,Transcript,SPLICE_SITE,5,"splice,intron",...,7,36,843,NaN,0.99521,0.924,NaN,NaN,2.271896,20.600
8610,X,154965965,C,T,SNV,0,Transcript,SPLICE_SITE,5,"splice,intron",...,7,36,843,NaN,0.98832,0.926,NaN,NaN,2.302688,20.800
8611,X,155492384,C,A,SNV,0,CodingTranscript,NON_SYNONYMOUS,7,missense,...,3,3,19,NaN,NaN,NaN,NaN,NaN,2.463914,21.700
8612,X,155492384,C,A,SNV,0,Transcript,INTRONIC,2,"intron,non_coding",...,3,3,19,NaN,NaN,NaN,NaN,NaN,2.463914,21.700


Below is the process used to export files for annotation for the coding region variants for HGMD for CADD:

In [39]:
hgmd_coding_region_variants_only_for_cadd = hgmd_disease_related_variants_mapped_to_nORFs_no_r_no_fp_no_dfp_no_dp_coding_region.drop(['INFO', 'QUAL', 'FILTER'], axis = 1)
hgmd_coding_region_variants_only_for_cadd

,CHROM,POS,ID,REF,ALT
0,chr10,100154922,CM140970,G,A
1,chr10,100183802,CM140971,C,A
2,chr10,100253438,CI1824020,A,AT
3,chr10,100256298,CD162836,TG,T
...,...,...,...,...,...
59426,chrX,9760731,CM981395,A,G
59427,chrX,9760732,CI183806,G,GA
59428,chrX,9760736,CD171619,GC,G
59429,chrX,9760741,CI115195,A,AG


In [39]:
header = """##fileformat=VCFv4.1
##fileDate=20090805
##source=myImputationProgramV3.1
##reference=file:///seq/references/
#CHROM POS ID REF ALT QUAL FILTER INFO
"""

output_VCF = "hgmd_coding_for_cadd.vcf"
with open(output_VCF, 'w') as vcf:
    vcf.write(header)

hgmd_coding_region_variants_only_for_cadd.to_csv(output_VCF, sep="\t", mode='a', index=False)

Below is the annotated file for the HGMD coding region for CADD:

In [33]:
hgmd_coding_annotated_cadd = pd.read_table('hgmd_coding_noheader.tsv', sep='\t')
hgmd_coding_annotated_cadd

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.
/opt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,120) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,#Chrom,Pos,Ref,Alt,Type,Length,AnnoType,Consequence,ConsScore,ConsDetail,...,Freq10000bp,Rare10000bp,Sngl10000bp,EnsembleRegulatoryFeature,dbscSNV-ada_score,dbscSNV-rf_score,RemapOverlapTF,RemapOverlapCL,RawScore,PHRED
0,1,1014143,C,T,SNV,0,CodingTranscript,STOP_GAINED,8,stop_gained,...,22,87,1622,Promoter,NaN,NaN,75.0,107.0,5.410558,35.0
1,1,1014143,C,T,SNV,0,Intergenic,UPSTREAM,1,upstream,...,22,87,1622,Promoter,NaN,NaN,75.0,107.0,5.410558,35.0
2,1,1014143,C,T,SNV,0,RegulatoryFeature,REGULATORY,4,regulatory,...,22,87,1622,Promoter,NaN,NaN,75.0,107.0,5.410558,35.0
3,1,1014143,C,T,SNV,0,RegulatoryFeature,REGULATORY,4,regulatory,...,22,87,1622,Promoter,NaN,NaN,75.0,107.0,5.410558,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89254,X,154966616,CTTCA,C,DEL,4,CodingTranscript,FRAME_SHIFT,7,frameshift,...,7,35,840,NaN,NaN,NaN,NaN,NaN,2.413846,21.5
89255,X,154966617,TTC,T,DEL,2,CodingTranscript,FRAME_SHIFT,7,frameshift,...,7,35,840,NaN,NaN,NaN,NaN,NaN,2.392576,21.3
89256,X,155524585,G,A,SNV,0,CodingTranscript,STOP_GAINED,8,stop_gained,...,9,26,682,NaN,NaN,NaN,4.0,4.0,6.231926,35.0
89257,X,155524585,G,A,SNV,0,Transcript,INTRONIC,2,"intron,non_coding",...,9,26,682,NaN,NaN,NaN,4.0,4.0,6.231926,35.0


## ClinVar

##### FOR ALL CLINVAR:

**I made the CHROM column values begin with chrchr# on accident. It should be set as chr# only. Therefore, I needed to change this. The documentation to do so is found under the ClinVar section of the "3)annotation using CADD script" section. I removed the string 'chrchr', cut out the column names row, and added the string 'chr.' I did this for every ClinVar dataframe here since they all share the same format.**

### CADD:

Below is the process used to export files for annotation for the non-coding regions for pathogenic variants from ClinVar for CADD:

In [49]:
clinvar_noncoding_region_pathogenic_variants_only_for_cadd = clinvar_noncoding_region_pathogenic_variants_only.drop(['INFO', 'QUAL', 'FILTER'], axis = 1)
clinvar_noncoding_region_pathogenic_variants_only_for_cadd

,CHROM,POS,ID,REF,ALT
38,chrchr10,100988540,225837,CT,C
51,chrchr10,100989084,488187,C,A
54,chrchr10,100989118,4628,G,A
56,chrchr10,100989154,4620,G,T
...,...,...,...,...,...
121537,chrchrX,74529428,212188,G,GC
121688,chrchrX,77902641,39768,A,G
121696,chrchrX,78003237,11786,G,A
121713,chrchrX,78122954,9955,G,A


In [35]:
header = """##fileformat=VCFv4.1
##fileDate=20090805
##source=myImputationProgramV3.1
##reference=file:///seq/references/
#CHROM POS ID REF ALT QUAL FILTER INFO
"""

output_VCF = "clinvar_noncoding_pathogenic_for_cadd.vcf"
with open(output_VCF, 'w') as vcf:
    vcf.write(header)

clinvar_noncoding_region_pathogenic_variants_only_for_cadd.to_csv(output_VCF, sep="\t", mode='a', index=False)

Below is the annotated file for the ClinVar non-coding region pathogenic variants for CADD:

In [63]:
clinvar_noncoding_pathogenic_annotated = pd.read_table('clinvar_noncoding_pathogenic_noheader.tsv', sep='\t')
clinvar_noncoding_pathogenic_annotated

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,#Chrom,Pos,Ref,Alt,Type,Length,AnnoType,Consequence,ConsScore,ConsDetail,...,Freq10000bp,Rare10000bp,Sngl10000bp,EnsembleRegulatoryFeature,dbscSNV-ada_score,dbscSNV-rf_score,RemapOverlapTF,RemapOverlapCL,RawScore,PHRED
0,1,11960768,G,A,SNV,0,RegulatoryFeature,REGULATORY,4,regulatory,...,21,90,1434,Promoter Flanking Region,0.99998,0.942,27.0,36.0,5.268674,35.0
1,1,11960768,G,A,SNV,0,Transcript,CANONICAL_SPLICE,6,splice_donor,...,21,90,1434,Promoter Flanking Region,0.99998,0.942,27.0,36.0,5.268674,35.0
2,1,11964787,T,C,SNV,0,Transcript,CANONICAL_SPLICE,6,splice_donor,...,19,99,1332,NaN,0.99987,0.886,10.0,10.0,4.686023,31.0
3,1,11972977,C,T,SNV,0,CodingTranscript,STOP_GAINED,8,stop_gained,...,20,95,1304,Promoter Flanking Region,NaN,NaN,19.0,23.0,7.497634,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2641,X,154419746,G,T,SNV,0,CodingTranscript,STOP_GAINED,8,"splice,stop_gained",...,11,29,822,NaN,0.99999,1.0,9.0,10.0,7.476349,38.0
2642,X,154419746,G,T,SNV,0,Intergenic,DOWNSTREAM,1,downstream,...,11,29,822,NaN,0.99999,1.0,9.0,10.0,7.476349,38.0
2643,X,154420211,G,C,SNV,0,Intergenic,DOWNSTREAM,1,downstream,...,12,32,828,NaN,0.99999,0.944,38.0,46.0,4.640280,29.8
2644,X,154420211,G,C,SNV,0,Transcript,CANONICAL_SPLICE,6,splice_acceptor,...,12,32,828,NaN,0.99999,0.944,38.0,46.0,4.640280,29.8


Below is the process used to export files for annotation for the non-coding regions for benign variants from ClinVar for CADD:

In [61]:
clinvar_noncoding_region_benign_variants_only_for_cadd = clinvar_noncoding_region_benign_variants_only.drop(['INFO', 'QUAL', 'FILTER'], axis = 1)
clinvar_noncoding_region_benign_variants_only_for_cadd

,CHROM,POS,ID,REF,ALT
73,chrchr10,100989312,136588,G,A
102,chrchr10,100990864,136589,C,T
103,chrchr10,100990866,136590,T,C
108,chrchr10,100991026,136591,C,A
...,...,...,...,...,...
121408,chrchrX,71132767,213614,CCTCTTCTCTTCTCTTCTCTTCTCTT,C
121410,chrchrX,71132767,95249,CCTCTT,C
121412,chrchrX,71132767,95251,CCTCTTCTCTTCTCTTCTCTTCTCTTCTCTT,C
121709,chrchrX,78118027,558817,C,T


In [62]:
header = """##fileformat=VCFv4.1
##fileDate=20090805
##source=myImputationProgramV3.1
##reference=file:///seq/references/
#CHROM POS ID REF ALT QUAL FILTER INFO
"""

output_VCF = "clinvar_noncoding_benign_for_cadd.vcf"
with open(output_VCF, 'w') as vcf:
    vcf.write(header)

clinvar_noncoding_region_benign_variants_only_for_cadd.to_csv(output_VCF, sep="\t", mode='a', index=False)

Below is the annotated file for the ClinVar non-coding region benign variants for CADD:

In [66]:
clinvar_noncoding_pathogenic_annotated = pd.read_table('clinvar_noncoding_benign_noheader.tsv', sep='\t')
clinvar_noncoding_pathogenic_annotated

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,#Chrom,Pos,Ref,Alt,Type,Length,AnnoType,Consequence,ConsScore,ConsDetail,...,Freq10000bp,Rare10000bp,Sngl10000bp,EnsembleRegulatoryFeature,dbscSNV-ada_score,dbscSNV-rf_score,RemapOverlapTF,RemapOverlapCL,RawScore,PHRED
0,1,1041950,T,C,SNV,0,Transcript,SPLICE_SITE,5,"splice,intron",...,23,100,1931,NaN,0.00001,0.002,10.0,13.0,0.092155,2.700
1,1,1042190,G,A,SNV,0,Transcript,INTRONIC,2,intron,...,25,109,1955,NaN,NaN,NaN,12.0,14.0,-0.401276,0.118
2,1,1043223,CCT,C,DEL,2,Transcript,INTRONIC,2,intron,...,24,117,2004,NaN,NaN,NaN,8.0,9.0,-0.012669,1.640
3,1,1045707,A,G,SNV,0,Transcript,INTRONIC,2,intron,...,22,131,2126,NaN,NaN,NaN,9.0,10.0,-0.057159,1.287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3200,X,154419685,C,T,SNV,0,Transcript,INTRONIC,2,intron,...,11,29,820,NaN,NaN,NaN,4.0,4.0,-0.033909,1.464
3201,X,154420108,C,T,SNV,0,Intergenic,DOWNSTREAM,1,downstream,...,12,31,826,NaN,NaN,NaN,36.0,47.0,-0.549155,0.036
3202,X,154420108,C,T,SNV,0,Transcript,INTRONIC,2,intron,...,12,31,826,NaN,NaN,NaN,36.0,47.0,-0.549155,0.036
3203,X,154961190,A,G,SNV,0,Transcript,INTRONIC,2,intron,...,7,45,823,NaN,NaN,NaN,1.0,1.0,0.351934,5.879


Below is the process used to export files for annotation for the coding regions for pathogenic coding variants from ClinVar for CADD:

In [64]:
clinvar_coding_region_pathogenic_variants_only_for_cadd = clinvar_coding_region_pathogenic_variants_only.drop(['INFO', 'QUAL', 'FILTER'], axis = 1)
clinvar_coding_region_pathogenic_variants_only_for_cadd

,CHROM,POS,ID,REF,ALT
1,chrchr10,100154922,226426,G,A
3,chrchr10,100183802,226427,C,A
7,chrchr10,100246864,504028,AT,A
11,chrchr10,100253422,419610,G,A
...,...,...,...,...,...
101084,chrchrX,85981796,279771,C,A
101094,chrchrX,9759332,10517,C,T
101098,chrchrX,9759390,10516,A,G
101099,chrchrX,9759390,10519,A,T


In [65]:
header = """##fileformat=VCFv4.1
##fileDate=20090805
##source=myImputationProgramV3.1
##reference=file:///seq/references/
#CHROM POS ID REF ALT QUAL FILTER INFO
"""

output_VCF = "clinvar_coding_pathogenic_for_cadd.vcf"
with open(output_VCF, 'w') as vcf:
    vcf.write(header)

clinvar_coding_region_pathogenic_variants_only_for_cadd.to_csv(output_VCF, sep="\t", mode='a', index=False)

Below is the annotated file for the ClinVar coding region pathogenic variants for CADD:

In [67]:
clinvar_coding_pathogenic_annotated = pd.read_table('clinvar_coding_pathogenic_noheader.tsv', sep='\t')
clinvar_coding_pathogenic_annotated

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.
/opt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,120) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,#Chrom,Pos,Ref,Alt,Type,Length,AnnoType,Consequence,ConsScore,ConsDetail,...,Freq10000bp,Rare10000bp,Sngl10000bp,EnsembleRegulatoryFeature,dbscSNV-ada_score,dbscSNV-rf_score,RemapOverlapTF,RemapOverlapCL,RawScore,PHRED
0,1,1014143,C,T,SNV,0,CodingTranscript,STOP_GAINED,8.0,stop_gained,...,22,87,1622,Promoter,NaN,NaN,75.0,107.0,5.410558,35.00
1,1,1014143,C,T,SNV,0,Intergenic,UPSTREAM,1.0,upstream,...,22,87,1622,Promoter,NaN,NaN,75.0,107.0,5.410558,35.00
2,1,1014143,C,T,SNV,0,RegulatoryFeature,REGULATORY,4.0,regulatory,...,22,87,1622,Promoter,NaN,NaN,75.0,107.0,5.410558,35.00
3,1,1014143,C,T,SNV,0,RegulatoryFeature,REGULATORY,4.0,regulatory,...,22,87,1622,Promoter,NaN,NaN,75.0,107.0,5.410558,35.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31074,X,155280059,G,C,SNV,0,CodingTranscript,NON_SYNONYMOUS,7.0,missense,...,11,33,827,Promoter Flanking Region,NaN,NaN,NaN,NaN,1.198603,13.85
31075,X,155280059,G,C,SNV,0,RegulatoryFeature,REGULATORY,4.0,regulatory,...,11,33,827,Promoter Flanking Region,NaN,NaN,NaN,NaN,1.198603,13.85
31076,X,155506930,GAT,G,DEL,2,CodingTranscript,FRAME_SHIFT,7.0,frameshift,...,6,39,652,NaN,NaN,NaN,1.0,1.0,4.018377,25.80
31077,X,155506930,GAT,G,DEL,2,Transcript,INTRONIC,2.0,"intron,non_coding",...,6,39,652,NaN,NaN,NaN,1.0,1.0,4.018377,25.80


Below is the process used to export files for annotation for the coding regions for benign coding variants from ClinVar for CADD:

In [68]:
clinvar_coding_region_benign_variants_only_for_cadd = clinvar_coding_region_benign_variants_only.drop(['INFO', 'QUAL', 'FILTER'], axis = 1)
clinvar_coding_region_benign_variants_only_for_cadd

,CHROM,POS,ID,REF,ALT
18,chrchr10,100987606,136593,G,T
57,chrchr10,102065918,284200,C,G
82,chrchr10,102396271,474786,G,A
87,chrchr10,102399466,541631,C,T
...,...,...,...,...,...
101076,chrchrX,85964016,255991,T,C
101079,chrchrX,85978770,497462,G,C
101081,chrchrX,85978816,377662,T,A
101087,chrchrX,93671995,208906,G,C


In [69]:
header = """##fileformat=VCFv4.1
##fileDate=20090805
##source=myImputationProgramV3.1
##reference=file:///seq/references/
#CHROM POS ID REF ALT QUAL FILTER INFO
"""

output_VCF = "clinvar_coding_benign_for_cadd.vcf"
with open(output_VCF, 'w') as vcf:
    vcf.write(header)

clinvar_coding_region_benign_variants_only_for_cadd.to_csv(output_VCF, sep="\t", mode='a', index=False)

Below is the annotated file for the ClinVar coding region benign variants for CADD:

In [86]:
clinvar_coding_benign_annotated = pd.read_table('clinvar_coding_benign_noheader.tsv', sep='\t')
clinvar_coding_benign_annotated

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.
/opt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,120) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,#Chrom,Pos,Ref,Alt,Type,Length,AnnoType,Consequence,ConsScore,ConsDetail,...,Freq10000bp,Rare10000bp,Sngl10000bp,EnsembleRegulatoryFeature,dbscSNV-ada_score,dbscSNV-rf_score,RemapOverlapTF,RemapOverlapCL,RawScore,PHRED
0,1,1014042,G,A,SNV,0,CodingTranscript,NON_SYNONYMOUS,7,missense,...,22,87,1616,Promoter,NaN,NaN,69.0,97.0,0.076262,2.519
1,1,1014042,G,A,SNV,0,Intergenic,UPSTREAM,1,upstream,...,22,87,1616,Promoter,NaN,NaN,69.0,97.0,0.076262,2.519
2,1,1014042,G,A,SNV,0,RegulatoryFeature,REGULATORY,4,regulatory,...,22,87,1616,Promoter,NaN,NaN,69.0,97.0,0.076262,2.519
3,1,1014042,G,A,SNV,0,RegulatoryFeature,REGULATORY,4,regulatory,...,22,87,1616,Promoter,NaN,NaN,69.0,97.0,0.076262,2.519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9134,X,154776813,C,CAAG,INS,3,CodingTranscript,INFRAME,6,inframe_insertion,...,4,22,980,NaN,NaN,NaN,7.0,7.0,0.893695,10.880
9135,X,154776813,C,CAAG,INS,3,Intergenic,DOWNSTREAM,1,downstream,...,4,22,980,NaN,NaN,NaN,7.0,7.0,0.893695,10.880
9136,X,154776813,C,CAAG,INS,3,Intergenic,DOWNSTREAM,1,downstream,...,4,22,980,NaN,NaN,NaN,7.0,7.0,0.893695,10.880
9137,X,154929926,T,G,SNV,0,CodingTranscript,SYNONYMOUS,5,synonymous,...,3,31,878,NaN,NaN,NaN,NaN,NaN,0.089486,2.669


## Human Derived 

### CADD:

Below is the process used to export files for annotation for the noncoding regions from the Human Derived data set:

Since there are already only 5 columns for this dataset, we do not have to code anything to drop any columns as we have had to do with the other dataframes.

human_derived_noncoding_region_variants_only - File we will use for our annotations for CADD.

In [17]:
human_derived_noncoding_region_variants_only

,CHROM,POS,ID,REF,ALT
0,chr10,1000013,.,G,A
1,chr10,100020652,.,G,A
2,chr10,1000297,.,T,G
3,chr10,1000555,.,A,T
...,...,...,...,...,...
1020168,chrX,9931817,.,T,C
1020169,chrX,9931818,.,G,A
1020170,chrX,9931993,.,T,C
1020171,chrX,9932000,.,C,T


Since this file is so large, I will break it into 10 separate pieces, each containing about 90,000 variants. The  code for this is shown below:

In [31]:
human_derived_noncoding_region_variants_only_1 = human_derived_noncoding_region_variants_only[:90000]
human_derived_noncoding_region_variants_only_1

,CHROM,POS,ID,REF,ALT
0,chr10,1000013,.,G,A
1,chr10,100020652,.,G,A
2,chr10,1000297,.,T,G
3,chr10,1000555,.,A,T
...,...,...,...,...,...
94277,chr11,47312241,.,G,A
94278,chr11,47312402,.,G,C
94279,chr11,47312428,.,C,T
94280,chr11,47312687,.,T,C


In [32]:
header = """##fileformat=VCFv4.1
##fileDate=20090805
##source=myImputationProgramV3.1
##reference=file:///seq/references/
#CHROM POS ID REF ALT QUAL FILTER INFO
"""

output_VCF = "human_derived_noncoding_for_cadd_1.vcf"
with open(output_VCF, 'w') as vcf:
    vcf.write(header)

human_derived_noncoding_region_variants_only_1.to_csv(output_VCF, sep="\t", mode='a', index=False)

In [33]:
human_derived_noncoding_region_variants_only_2 = human_derived_noncoding_region_variants_only[90000:180000]
human_derived_noncoding_region_variants_only_2

,CHROM,POS,ID,REF,ALT
94281,chr11,47312814,.,G,A
94282,chr11,47312979,.,T,G
94283,chr11,47313760,.,G,T
94284,chr11,47313853,.,A,G
...,...,...,...,...,...
190691,chr12,65348795,.,C,A
190692,chr12,65348797,.,C,T
190693,chr12,65348836,.,C,T
190694,chr12,65348914,.,C,T


In [44]:
header = """##fileformat=VCFv4.1
##fileDate=20090805
##source=myImputationProgramV3.1
##reference=file:///seq/references/
#CHROM POS ID REF ALT QUAL FILTER INFO
"""

output_VCF = "human_derived_noncoding_for_cadd_2.vcf"
with open(output_VCF, 'w') as vcf:
    vcf.write(header)

human_derived_noncoding_region_variants_only_2.to_csv(output_VCF, sep="\t", mode='a', index=False)

In [34]:
human_derived_noncoding_region_variants_only_3 = human_derived_noncoding_region_variants_only[180000:270000]

In [45]:
header = """##fileformat=VCFv4.1
##fileDate=20090805
##source=myImputationProgramV3.1
##reference=file:///seq/references/
#CHROM POS ID REF ALT QUAL FILTER INFO
"""

output_VCF = "human_derived_noncoding_for_cadd_3.vcf"
with open(output_VCF, 'w') as vcf:
    vcf.write(header)

human_derived_noncoding_region_variants_only_3.to_csv(output_VCF, sep="\t", mode='a', index=False)

In [35]:
human_derived_noncoding_region_variants_only_4 = human_derived_noncoding_region_variants_only[270000:360000]

In [46]:
header = """##fileformat=VCFv4.1
##fileDate=20090805
##source=myImputationProgramV3.1
##reference=file:///seq/references/
#CHROM POS ID REF ALT QUAL FILTER INFO
"""

output_VCF = "human_derived_noncoding_for_cadd_4.vcf"
with open(output_VCF, 'w') as vcf:
    vcf.write(header)

human_derived_noncoding_region_variants_only_4.to_csv(output_VCF, sep="\t", mode='a', index=False)

In [36]:
human_derived_noncoding_region_variants_only_5 = human_derived_noncoding_region_variants_only[360000:450000]

In [47]:
header = """##fileformat=VCFv4.1
##fileDate=20090805
##source=myImputationProgramV3.1
##reference=file:///seq/references/
#CHROM POS ID REF ALT QUAL FILTER INFO
"""

output_VCF = "human_derived_noncoding_for_cadd_5.vcf"
with open(output_VCF, 'w') as vcf:
    vcf.write(header)

human_derived_noncoding_region_variants_only_5.to_csv(output_VCF, sep="\t", mode='a', index=False)

In [37]:
human_derived_noncoding_region_variants_only_6 = human_derived_noncoding_region_variants_only[450000:540000]

In [48]:
header = """##fileformat=VCFv4.1
##fileDate=20090805
##source=myImputationProgramV3.1
##reference=file:///seq/references/
#CHROM POS ID REF ALT QUAL FILTER INFO
"""

output_VCF = "human_derived_noncoding_for_cadd_6.vcf"
with open(output_VCF, 'w') as vcf:
    vcf.write(header)

human_derived_noncoding_region_variants_only_6.to_csv(output_VCF, sep="\t", mode='a', index=False)

In [38]:
human_derived_noncoding_region_variants_only_7 = human_derived_noncoding_region_variants_only[540000:630000]

In [49]:
header = """##fileformat=VCFv4.1
##fileDate=20090805
##source=myImputationProgramV3.1
##reference=file:///seq/references/
#CHROM POS ID REF ALT QUAL FILTER INFO
"""

output_VCF = "human_derived_noncoding_for_cadd_7.vcf"
with open(output_VCF, 'w') as vcf:
    vcf.write(header)

human_derived_noncoding_region_variants_only_7.to_csv(output_VCF, sep="\t", mode='a', index=False)

In [39]:
human_derived_noncoding_region_variants_only_8 = human_derived_noncoding_region_variants_only[630000:720000]

In [50]:
header = """##fileformat=VCFv4.1
##fileDate=20090805
##source=myImputationProgramV3.1
##reference=file:///seq/references/
#CHROM POS ID REF ALT QUAL FILTER INFO
"""

output_VCF = "human_derived_noncoding_for_cadd_8.vcf"
with open(output_VCF, 'w') as vcf:
    vcf.write(header)

human_derived_noncoding_region_variants_only_8.to_csv(output_VCF, sep="\t", mode='a', index=False)

In [51]:
human_derived_noncoding_region_variants_only_9 = human_derived_noncoding_region_variants_only[720000:810000]

In [52]:
header = """##fileformat=VCFv4.1
##fileDate=20090805
##source=myImputationProgramV3.1
##reference=file:///seq/references/
#CHROM POS ID REF ALT QUAL FILTER INFO
"""

output_VCF = "human_derived_noncoding_for_cadd_9.vcf"
with open(output_VCF, 'w') as vcf:
    vcf.write(header)

human_derived_noncoding_region_variants_only_9.to_csv(output_VCF, sep="\t", mode='a', index=False)

In [41]:
human_derived_noncoding_region_variants_only_10 = human_derived_noncoding_region_variants_only[810000:900000]
human_derived_noncoding_region_variants_only_10

,CHROM,POS,ID,REF,ALT
855720,chr7,107252617,.,A,T
855721,chr7,107252854,.,T,C
855722,chr7,107252864,.,A,T
855723,chr7,107283879,.,C,T
...,...,...,...,...,...
949468,chr8,47381618,.,T,C
949469,chr8,47381868,.,A,C
949470,chr8,47382570,.,A,G
949471,chr8,47382847,.,A,C


In [53]:
header = """##fileformat=VCFv4.1
##fileDate=20090805
##source=myImputationProgramV3.1
##reference=file:///seq/references/
#CHROM POS ID REF ALT QUAL FILTER INFO
"""

output_VCF = "human_derived_noncoding_for_cadd_10.vcf"
with open(output_VCF, 'w') as vcf:
    vcf.write(header)

human_derived_noncoding_region_variants_only_10.to_csv(output_VCF, sep="\t", mode='a', index=False)

In [43]:
human_derived_noncoding_region_variants_only_11 = human_derived_noncoding_region_variants_only[900000:]
human_derived_noncoding_region_variants_only_11

,CHROM,POS,ID,REF,ALT
949472,chr8,47382972,.,A,G
949473,chr8,47382981,.,T,C
949474,chr8,47383038,.,A,G
949475,chr8,47383524,.,G,A
...,...,...,...,...,...
1020168,chrX,9931817,.,T,C
1020169,chrX,9931818,.,G,A
1020170,chrX,9931993,.,T,C
1020171,chrX,9932000,.,C,T


In [54]:
header = """##fileformat=VCFv4.1
##fileDate=20090805
##source=myImputationProgramV3.1
##reference=file:///seq/references/
#CHROM POS ID REF ALT QUAL FILTER INFO
"""

output_VCF = "human_derived_noncoding_for_cadd_11.vcf"
with open(output_VCF, 'w') as vcf:
    vcf.write(header)

human_derived_noncoding_region_variants_only_11.to_csv(output_VCF, sep="\t", mode='a', index=False)

Below are the annotated files for the human derived non-coding region variants for CADD:

In [74]:
human_derived_noncoding_annotated_1 = pd.read_table('human_derived_noncoding_for_cadd_1_noheader.tsv', sep='\t')
human_derived_noncoding_annotated_1

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.
/opt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (16,17,30,35,37,120) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,#Chrom,Pos,Ref,Alt,Type,Length,AnnoType,Consequence,ConsScore,ConsDetail,...,Freq10000bp,Rare10000bp,Sngl10000bp,EnsembleRegulatoryFeature,dbscSNV-ada_score,dbscSNV-rf_score,RemapOverlapTF,RemapOverlapCL,RawScore,PHRED
0,1,1001313,G,T,SNV,0,RegulatoryFeature,REGULATORY,4,regulatory,...,25.0,88.0,1931.0,Promoter,NaN,NaN,60.0,90.0,0.053718,2.275
1,1,1001313,G,T,SNV,0,RegulatoryFeature,REGULATORY,4,regulatory,...,25.0,88.0,1931.0,Promoter,NaN,NaN,60.0,90.0,0.053718,2.275
2,1,1001313,G,T,SNV,0,Intergenic,UPSTREAM,1,upstream,...,25.0,88.0,1931.0,Promoter,NaN,NaN,60.0,90.0,0.053718,2.275
3,1,1001313,G,T,SNV,0,Transcript,INTRONIC,2,intron,...,25.0,88.0,1931.0,Promoter,NaN,NaN,60.0,90.0,0.053718,2.275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135092,11,134383329,G,C,SNV,0,Intergenic,DOWNSTREAM,1,downstream,...,23.0,62.0,1401.0,NaN,NaN,NaN,NaN,NaN,-0.154648,0.721
135093,11,134383329,G,C,SNV,0,Transcript,INTRONIC,2,intron,...,23.0,62.0,1401.0,NaN,NaN,NaN,NaN,NaN,-0.154648,0.721
135094,11,134383599,G,A,SNV,0,Transcript,INTRONIC,2,intron,...,24.0,59.0,1411.0,NaN,NaN,NaN,10.0,12.0,-0.097920,1.017
135095,11,134383614,T,C,SNV,0,Transcript,INTRONIC,2,intron,...,24.0,60.0,1412.0,NaN,NaN,NaN,10.0,12.0,0.049698,2.233


In [75]:
human_derived_noncoding_annotated_2 = pd.read_table('human_derived_noncoding_for_cadd_2_noheader.tsv', sep='\t')
human_derived_noncoding_annotated_2

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.
/opt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13,16,17,30,35,37,120) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,#Chrom,Pos,Ref,Alt,Type,Length,AnnoType,Consequence,ConsScore,ConsDetail,...,Freq10000bp,Rare10000bp,Sngl10000bp,EnsembleRegulatoryFeature,dbscSNV-ada_score,dbscSNV-rf_score,RemapOverlapTF,RemapOverlapCL,RawScore,PHRED
0,1,1668612,C,G,SNV,0,Transcript,INTRONIC,2,intron,...,20,80,1740,NaN,NaN,NaN,NaN,NaN,-0.161503,0.690
1,1,1668612,C,G,SNV,0,Intergenic,UPSTREAM,1,upstream,...,20,80,1740,NaN,NaN,NaN,NaN,NaN,-0.161503,0.690
2,1,1668740,G,A,SNV,0,Transcript,INTRONIC,2,intron,...,20,84,1743,NaN,NaN,NaN,NaN,NaN,-0.209266,0.506
3,1,1668740,G,A,SNV,0,Intergenic,UPSTREAM,1,upstream,...,20,84,1743,NaN,NaN,NaN,NaN,NaN,-0.209266,0.506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148269,12,133056532,T,C,SNV,0,Transcript,INTRONIC,2,intron,...,23,60,1336,NaN,NaN,NaN,2.0,2.0,-0.331295,0.206
148270,12,133056612,G,A,SNV,0,Transcript,INTRONIC,2,intron,...,23,59,1330,NaN,NaN,NaN,3.0,3.0,-0.448011,0.081
148271,12,133056705,T,C,SNV,0,Transcript,INTRONIC,2,intron,...,21,59,1337,NaN,NaN,NaN,3.0,4.0,-0.019107,1.585
148272,12,133056935,G,T,SNV,0,Transcript,INTRONIC,2,intron,...,21,59,1336,NaN,NaN,NaN,5.0,6.0,-0.231915,0.433


In [76]:
human_derived_noncoding_annotated_3 = pd.read_table('human_derived_noncoding_for_cadd_3_noheader.tsv', sep='\t')
human_derived_noncoding_annotated_3

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.
/opt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (35,37,120) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,#Chrom,Pos,Ref,Alt,Type,Length,AnnoType,Consequence,ConsScore,ConsDetail,...,Freq10000bp,Rare10000bp,Sngl10000bp,EnsembleRegulatoryFeature,dbscSNV-ada_score,dbscSNV-rf_score,RemapOverlapTF,RemapOverlapCL,RawScore,PHRED
0,1,3385306,A,G,SNV,0,Transcript,INTRONIC,2,intron,...,30,95,1620,NaN,NaN,NaN,3.0,4.0,0.105947,2.863
1,1,3385442,T,C,SNV,0,Transcript,INTRONIC,2,intron,...,29,95,1626,NaN,NaN,NaN,6.0,7.0,0.097632,2.764
2,1,3385843,T,C,SNV,0,Transcript,INTRONIC,2,intron,...,29,91,1637,NaN,NaN,NaN,8.0,10.0,-0.120348,0.890
3,1,3385881,G,C,SNV,0,Transcript,INTRONIC,2,intron,...,29,91,1636,NaN,NaN,NaN,8.0,10.0,-0.299183,0.264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146449,15,101781876,C,T,SNV,0,Transcript,INTRONIC,2,"intron,non_coding",...,14,96,1738,Promoter Flanking Region,NaN,NaN,2.0,2.0,-0.300977,0.261
146450,15,101781883,C,G,SNV,0,RegulatoryFeature,REGULATORY,4,regulatory,...,14,96,1739,Promoter Flanking Region,NaN,NaN,2.0,2.0,0.225834,4.395
146451,15,101781883,C,G,SNV,0,Transcript,INTRONIC,2,"intron,non_coding",...,14,96,1739,Promoter Flanking Region,NaN,NaN,2.0,2.0,0.225834,4.395
146452,15,101781883,C,G,SNV,0,Transcript,INTRONIC,2,"intron,non_coding",...,14,96,1739,Promoter Flanking Region,NaN,NaN,2.0,2.0,0.225834,4.395


In [77]:
human_derived_noncoding_annotated_4 = pd.read_table('human_derived_noncoding_for_cadd_4_noheader.tsv', sep='\t')
human_derived_noncoding_annotated_4

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.
/opt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (16,17,30,35,37,120) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,#Chrom,Pos,Ref,Alt,Type,Length,AnnoType,Consequence,ConsScore,ConsDetail,...,Freq10000bp,Rare10000bp,Sngl10000bp,EnsembleRegulatoryFeature,dbscSNV-ada_score,dbscSNV-rf_score,RemapOverlapTF,RemapOverlapCL,RawScore,PHRED
0,1,5875122,T,C,SNV,0,Transcript,INTRONIC,2,intron,...,31,84,1744,NaN,NaN,NaN,5.0,5.0,-0.188326,0.581
1,1,5875194,G,C,SNV,0,Transcript,INTRONIC,2,intron,...,30,85,1749,NaN,NaN,NaN,3.0,3.0,-0.517745,0.046
2,1,5875324,T,C,SNV,0,Transcript,INTRONIC,2,intron,...,30,86,1755,NaN,NaN,NaN,4.0,6.0,-0.343136,0.188
3,1,5875346,T,C,SNV,0,Transcript,INTRONIC,2,intron,...,30,86,1758,NaN,NaN,NaN,4.0,6.0,-0.074180,1.168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148917,17,48834602,C,T,SNV,0,Transcript,INTRONIC,2,intron,...,22,45,1296,NaN,NaN,NaN,2.0,3.0,0.578898,8.017
148918,17,48834679,G,A,SNV,0,Transcript,INTRONIC,2,intron,...,23,45,1294,NaN,NaN,NaN,2.0,2.0,0.811848,10.050
148919,17,48834896,C,A,SNV,0,Transcript,INTRONIC,2,intron,...,24,44,1308,NaN,NaN,NaN,2.0,2.0,-0.007444,1.685
148920,17,48834902,G,C,SNV,0,Transcript,INTRONIC,2,intron,...,24,44,1308,NaN,NaN,NaN,3.0,3.0,-0.027706,1.514


In [ ]:
human_derived_noncoding_annotated_5 = pd.read_table('human_derived_noncoding_for_cadd_5_noheader.tsv', sep='\t')
human_derived_noncoding_annotated_5

In [ ]:
human_derived_noncoding_annotated_6 = pd.read_table('human_derived_noncoding_for_cadd_6_noheader.tsv', sep='\t')
human_derived_noncoding_annotated_6

In [ ]:
human_derived_noncoding_annotated_7 = pd.read_table('human_derived_noncoding_for_cadd_7_noheader.tsv', sep='\t')
human_derived_noncoding_annotated_7

In [ ]:
human_derived_noncoding_annotated_8 = pd.read_table('human_derived_noncoding_for_cadd_8_noheader.tsv', sep='\t')
human_derived_noncoding_annotated_8

In [ ]:
human_derived_noncoding_annotated_9 = pd.read_table('human_derived_noncoding_for_cadd_9_noheader.tsv', sep='\t')
human_derived_noncoding_annotated_9

In [ ]:
human_derived_noncoding_annotated_10 = pd.read_table('human_derived_noncoding_for_cadd_10_noheader.tsv', sep='\t')
human_derived_noncoding_annotated_10

In [ ]:
human_derived_noncoding_annotated_11 = pd.read_table('human_derived_noncoding_for_cadd_11_noheader.tsv', sep='\t')
human_derived_noncoding_annotated_11

Now we have to combine all of these files back into one complete dataframe to give the total annotated non-coding variants from the human derived data for CADD:

In [ ]:
human_derived_noncoding_annotated = pd.concat('human_derived_noncoding_annotated_1', 'human_derived_noncoding_annotated_2', 
                                             'human_derived_noncoding_annotated_3', 'human_derived_noncoding_annotated_4', 
                                             'human_derived_noncoding_annotated_5', 'human_derived_noncoding_annotated_6',
                                             'human_derived_noncoding_annotated_7', 'human_derived_noncoding_annotated_8',
                                             'human_derived_noncoding_annotated_9', 'human_derived_noncoding_annotated_10', 
                                             'human_derived_noncoding_annotated_11')
human_derived_noncoding_annotated

Below is the process used to export files for annotation for the coding regions from the Human Derived data set for CADD:

In [28]:
human_derived_coding_region_variants_only

,CHROM,POS,ID,REF,ALT
0,chr10,100020652,.,G,A
1,chr10,100190879,.,T,C
2,chr10,100233196,.,G,A
3,chr10,100267615,.,C,T
...,...,...,...,...,...
53794,chrX,99719539,.,A,G
53795,chrX,99719939,.,C,A
53796,chrX,99720084,.,G,A
53797,chrX,99721008,.,G,A


In [1]:
header = """##fileformat=VCFv4.1
##fileDate=20090805
##source=myImputationProgramV3.1
##reference=file:///seq/references/
#CHROM POS ID REF ALT QUAL FILTER INFO
"""

output_VCF = "human_derived_coding_for_cadd.vcf"
with open(output_VCF, 'w') as vcf:
    vcf.write(header)

human_derived_coding_region_variants_only.to_csv(output_VCF, sep="\t", mode='a', index=False)

NameError: name 'human_derived_coding_region_variants_only' is not defined

Below is the annotated file for the human derived coding region variants for CADD:

In [73]:
human_derived_coding_annotated = pd.read_table('human_derived_coding_noheader.tsv', sep='\t')
human_derived_coding_annotated

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.
/opt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,120) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,#Chrom,Pos,Ref,Alt,Type,Length,AnnoType,Consequence,ConsScore,ConsDetail,...,Freq10000bp,Rare10000bp,Sngl10000bp,EnsembleRegulatoryFeature,dbscSNV-ada_score,dbscSNV-rf_score,RemapOverlapTF,RemapOverlapCL,RawScore,PHRED
0,1,943329,C,T,SNV,0,RegulatoryFeature,REGULATORY,4,regulatory,...,18,117,2586,CTCF Binding Site,NaN,NaN,88.0,121.0,0.590702,8.117
1,1,943329,C,T,SNV,0,CodingTranscript,SYNONYMOUS,5,synonymous,...,18,117,2586,CTCF Binding Site,NaN,NaN,88.0,121.0,0.590702,8.117
2,1,943329,C,T,SNV,0,Intergenic,DOWNSTREAM,1,downstream,...,18,117,2586,CTCF Binding Site,NaN,NaN,88.0,121.0,0.590702,8.117
3,1,944699,C,T,SNV,0,CodingTranscript,NON_SYNONYMOUS,7,missense,...,16,117,2573,NaN,NaN,NaN,12.0,13.0,2.980499,23.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122229,X,155260197,C,A,SNV,0,Transcript,3PRIME_UTR,2,3_prime_UTR,...,9,36,873,NaN,NaN,NaN,3.0,6.0,-0.219204,0.473
122230,X,155612732,T,C,SNV,0,RegulatoryFeature,REGULATORY,4,regulatory,...,7,27,866,Promoter,NaN,NaN,112.0,202.0,0.661682,8.709
122231,X,155612732,T,C,SNV,0,Transcript,INTRONIC,2,intron,...,7,27,866,Promoter,NaN,NaN,112.0,202.0,0.661682,8.709
122232,X,155612732,T,C,SNV,0,Intergenic,DOWNSTREAM,1,downstream,...,7,27,866,Promoter,NaN,NaN,112.0,202.0,0.661682,8.709
